In [6]:
import cv2
import numpy as np
import imutils    
import time


In [7]:
def white_balance(img):
    result = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    avg_a = np.average(result[:, :, 1])
    avg_b = np.average(result[:, :, 2])
    result[:, :, 1] = result[:, :, 1] - ((avg_a - 128) * (result[:, :, 0] / 255.0) * 1.1)
    result[:, :, 2] = result[:, :, 2] - ((avg_b - 128) * (result[:, :, 0] / 255.0) * 1.1)
    result = cv2.cvtColor(result, cv2.COLOR_LAB2BGR)
    return result


def find_roi(frame):
    frame = white_balance(frame)
    circles = []
    
    lower_red = np.array([131,45,0], dtype=np.uint8) #153
    upper_red = np.array([255,255,158], dtype=np.uint8)
    
#     lower_red = np.array([153,30,0], dtype=np.uint8) #153
#     upper_red = np.array([255,255,255], dtype=np.uint8)
    
#     lower_red2 = np.array([121,16,0], dtype=np.uint8) #153
#     upper_red2 = np.array([255,255,255], dtype=np.uint8)
    
    lower_dark_red = np.array([131,31,0], dtype=np.uint8)
    upper_dark_red = np.array([174,255,255], dtype=np.uint8)
    
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    mask_red = cv2.inRange(hsv, lower_red, upper_red)
    mask_dark_red = cv2.inRange(hsv, lower_dark_red, upper_dark_red)
    mask = mask_red + mask_dark_red

    kernel = np.ones((4,4),np.uint8)
  
    dilate_img = cv2.dilate(mask,kernel,iterations = 3)
    erode = cv2.erode(dilate_img, kernel, iterations = 4)
    
    cnts = cv2.findContours(erode.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    cnts = imutils.grab_contours(cnts)
    for c in cnts:
        M = cv2.moments(c)
        if cv2.contourArea(c) > 100 and cv2.arcLength(c, True > 200):
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
            circles.append((cX,cY))
    
    circles.sort()
    roi = None
    if len(circles)==4:
        roi = circles[1][1],circles[0][1],circles[1][0],circles[2][0]
    return  roi, circles


    

In [8]:
def rected_text(img,rect,text,font=cv2.FONT_HERSHEY_SIMPLEX,fontScale=1.5,fontColor=(255,255,255),thickness=2,draw_border=True):
    x,y,w,h = rect
    height, width = img.shape[:2]
    text_size = cv2.getTextSize(text, font, fontScale, thickness)
    margin = int(10*fontScale)

    pos = [x,y]
    center = x+w/2,y+h/2
    if width/2 - center[0] < 0:
        pos[0] = x+w-(+text_size[0][0]+margin)
    else:
        pos[0] = x
    
    if height/2 - center[1] >= 0:
        pos[1] = y+h+(text_size[0][1]+margin*2)
    else:
        pos[1] = y
    
    pos = tuple(pos)
    pos2 = pos[0]+text_size[0][0]+margin,pos[1]-text_size[0][1]-margin*2
    
    if draw_border: cv2.rectangle(img,(x,y),(x+w,y+h),fontColor,2)
    cv2.rectangle(img,pos,pos2,(0,100,0),-1)
    cv2.rectangle(img,pos,pos2,fontColor,thickness)
    cv2.putText(img,text,(pos[0]+margin,pos[1]-margin),font,fontScale,fontColor,thickness,bottomLeftOrigin=False)
    
    
current_milli_time = lambda: int(round(time.time() * 1000))

In [ ]:
cap = cv2.VideoCapture(0)
fps, total_fps = 0, 0
last_frame = current_milli_time()
while True:
    
    ret,frame = cap.read()
    img = frame.copy()
       
    roi, circles = find_roi(frame)
    
    
    for x,y in circles:
        
        cv2.circle(img, (x, y), 10, (0,255,0), 2) 
        cv2.putText(img, f"{x}-{y}", (x+10,y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 1)
        
    
    fps += 1        
    rected_text(img,(0,0,0,0),f"FPS: {total_fps}",fontScale=0.5,thickness=1,draw_border=False)
    cv2.imshow("Image", img)
#     cv2.imshow("Erode", erode)
    if current_milli_time() - last_frame > 1000:
        total_fps = fps
        last_frame = current_milli_time()
        fps = 0 
    
    
    if roi is not None:
        img2 = frame.copy()
        x,y,w,z = roi

        try:
            img2 = img2[x:y,w:z]
            cv2.imshow("ROI",img2)
        except:
            continue
        
       
    if cv2.waitKey(1) & 0xFF == ord('q') or ret == False:
        break 

# cv2.imshow("Image", c_im)
# cv2.imshow("Image2", img2)
cv2.waitKey(0)
cap.release()
cv2.destroyAllWindows()

In [5]:
cap.release()